 # Initialization

In [ ]:
# Requirements
# Tested on Python 3.12.3
# !pip install -r requirements.txt

import matplotlib.pyplot as plt
plt.close('all')

from definition import *
from tools.log import LOGGER

In [ ]:
# Load the Environment - encodes the problem statement and serves as input to the offline preprocessing algorithm (See Tech Report Section 2.2.1)
env_file: str = './tasks/env_1_1.json'
env = Environment.from_json(filename=env_file)

# Offline Solution Generation
See Tech Report Section 3

### Task Preparation

In [ ]:
# Initialize Solution Container - contains all data generated by the offline preprocessing algorithm
sol = Solution(env=env)

# Initialize Planner - performs the algorithm, populating the solution
from offline.planner import Planner
planner = Planner(env=env, sol=sol)

### Flow Generation
See Tech Report Section 3.1

In [ ]:
use_alltoall: bool = False # Instead of minimal connectivity, connect all receivers to all providers and vice-versa

LOGGER.log_perf_start("Flow Generation")
planner.generate_flow(use_alltoall=use_alltoall)
LOGGER.log_perf_end()

### Alternative: Load Flow Network
Use already computed flow network, replacing Flow Generation

In [ ]:
if False:
    provided_flow_file: str = './output/flow.json'
    provided_flow: Network = Network.from_json(filename=provided_flow_file, env=env)
    planner.use_provided_flow(flow=provided_flow)

### Network Initialization
See Tech Report Section 3.2

In [ ]:
use_merge_split: bool = False # Replace cross intersections with merge-split intersections (Tech Report Section 2.4)
max_length: int | None = 5 # If not None, arcs above a certain length are split into sections by inserting corner primitives at equidistant points

LOGGER.log_perf_start("Network Initialization")
planner.initialize_network(use_merge_split=use_merge_split, max_length=max_length)
LOGGER.log_perf_end()

### Alternative: Load Network
Use already computed network, replacing both Flow Generation and Network Initialization

In [ ]:
if False:
    checkpoint_network_file: str = './output/iter_00.json'
    checkpoint_network: Network = Network.from_json(filename=checkpoint_network_file, env=env)
    planner.use_checkpoint_network(checkpoint_network)

### Network Optimization
See Tech Report Section 3.3

In [ ]:
draw_vid: bool = True # Generate visualization of optimization - Disable to increase performance
show_vid_freq: int | None = 0 # None: Do not show during execution, 0: Show at start of iteration, n>0: Show every n steps

# The optimization can be halted prematurely by sending a KeyboardInterrupt or interrupting the execution of this cell
LOGGER.log_perf_start("Optimization")
planner.optimize_network(draw_vid=draw_vid, show_vid_freq=show_vid_freq)
LOGGER.log_perf_end()

### Post-Processing
See Tech Report Section 3.4

In [ ]:
LOGGER.log_perf_start("Post-Processing")

# Optional - if all station flows are bidirectional, with each station either having no flow or sending and receiving items from any other station,
# the optimal matching is symmetric as well and can be reversed by flipping the directions of all paths. This can be performed at any stage until 
if False:
    if sol.reversible: planner.reverse_network()

planner.post_process()
planner.generate_graph()
planner.generate_item_sequences()
planner.export_solution()
LOGGER.log_perf_end()

### Alternative: Load Solution
Use already computed solution, replacing Offline Generation

In [ ]:
if True:
    solution_file: str = './output/sol.p'
    sol = Solution.from_pickle(filename=solution_file)
    env = sol.env
    sol = sol

# Simulation of Online Control
See Tech Report Section 4

### Simulator Setup
See Tech Report Section 4.1

In [ ]:
mover_num: int = 20
time_max: float = 60.0

# Generate Scenario - contains the initial environment specifications, available number of movers and runtime of the simulation
sce = Scenario(name="Scenario", env=env, mover_num=mover_num, time_max=time_max)

# Initialize Sequence - contains all data generated by the simulation
seq = Sequence(sol=sol, sce=sce)

# Initialize Simulator - performs the algorithm, populating the sequence
from online.simulator import Simulator
simulator = Simulator(sce=sce, sol=sol, seq=seq)

### Simulator Execution

In [ ]:
LOGGER.log_perf_start("Controller Simulation")
simulator.run()
LOGGER.log_perf_end()

LOGGER.log_perf_start("Sequence Export")
simulator.export_sequence()
LOGGER.log_perf_end()

### Alternative: Load Sequence
Use already computed sequence, replacing Simulation

In [ ]:
if False:
    sequence_file: str = './output/seq.p'
    seq = Sequence.from_pickle(filename=sequence_file)
    simulator = Simulator.from_sequence(seq=seq)

### Solution and Sequence Visualization

In [ ]:
from tools.gui import GUI
gui = GUI(env=env, sol=sol, seq=seq)
gui.run(quit_after_close = False) # quit_after_close = False is required for re-runs in the same ipynb instance